In [1]:
# PREPROCESSING CODE FOR IMAGING DATA with Behaviour
# Creates CSV, aligned the time events, runs suite2p and calculate dff for all cells
# Creates an analysis folder & saves necessary files for each recording session
# # This code creates necessary behaviour & imaging files
# It should work for any given task recorded in Dual2p or Packer 1 scope
# Requires MATLAB integration, and uses getBehavData matlab function
# 
# 30/08/2025 HA
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import sys,os, glob, shutil
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import plot_funcs as pfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
from scipy.stats import zscore
import pickle
import time
from LakLabAnalysis.Utility.extract_paq_events import extract_paq_data_frame
import LakLabAnalysis.Utility.utils_funcs as utils_laklab
from tifffile import TiffFile, imread, imwrite
import imageio.v2 as imageio

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')


Takes <30s. However, it might take >1 mins or so if network is busy...
Matlab engine is set correctly.


<div style="
  margin:14px 10px; padding:14px 18px;
  background:linear-gradient(90deg,#7c3aed,#06b6d4);
  color:white; font-size:24px; font-weight:800;
  border-radius:10px; letter-spacing:.3px;">
  ⬇️ 
  Preprocessing for GOOD performance + Imaged session
  </span>
</div>

In [38]:
# Some functions required below
def create_FOV_withSelectedCells(isCell, ops, s2p_path):
    prob_threshold = 0
    cell_indices = np.where((isCell[:,1] > prob_threshold))[0]

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 12))

    # Show mean image
    ax.imshow(ops['meanImg'], cmap='binary_r')

    # Load stat.npy to get cell coordinates
    stat = np.load(os.path.join(s2p_path, 'stat.npy'), allow_pickle=True)

    # Generate random colors for each cell
    colors = np.random.rand(len(cell_indices), 3)

    # Draw ROIs of filtered cells
    for idx, cell_number in enumerate(cell_indices):
        stat_cell = stat[cell_number]
        ypix = [stat_cell['ypix'][i] for i in range(len(stat_cell['ypix'])) if not stat_cell['overlap'][i]]
        xpix = [stat_cell['xpix'][i] for i in range(len(stat_cell['xpix'])) if not stat_cell['overlap'][i]]
        ax.plot(xpix, ypix, '.', markersize=1, alpha=0.7, color=colors[idx])

    ax.set_title(f"Detected cells (prob > {prob_threshold}): {len(cell_indices)}")
    ax.axis('off')
    plt.show()
    fig.savefig(
        os.path.join(info.recordingList.analysispathname[ind], f"MeanImG.png"),
        dpi=300,          # higher resolution
        bbox_inches='tight',  # trim whitespace
        pad_inches=0.1,       # small padding
        facecolor='white'     # ensure background matches
    )

    # Also show enhanced image (meanImgE) which might be clearer
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(ops['meanImgE'], cmap='binary_r')

    # Draw ROIs of filtered cells
    for idx, cell_number in enumerate(cell_indices):
        stat_cell = stat[cell_number]
        ypix = [stat_cell['ypix'][i] for i in range(len(stat_cell['ypix'])) if not stat_cell['overlap'][i]]
        xpix = [stat_cell['xpix'][i] for i in range(len(stat_cell['xpix'])) if not stat_cell['overlap'][i]]
        ax.plot(xpix, ypix, '.', markersize=1, alpha=0.7, color=colors[idx])

    ax.set_title(f"Detected cells (prob > {prob_threshold}): {len(cell_indices)}")
    ax.axis('off')
    # save this image in the analysis folder
    fig.savefig(
        os.path.join(info.recordingList.analysispathname[ind], f"FOV_withSelectedCells_{prob_threshold}.png"),
        dpi=300,          # higher resolution
        bbox_inches='tight',  # trim whitespace
        pad_inches=0.1,       # small padding
        facecolor='white'     # ensure background matches
    )
    #close the plot
    plt.close(fig)

def calculateHeatmapsForRewardedStimulus():
   ## Parameters
   fRate = 1000/30
   responsiveness_test_duration = 1000.0 #in ms 
   pre_frames    = 2000.0# in ms
   pre_frames    = int(np.ceil(pre_frames/fRate))
   post_frames   = 6000.0 # in ms
   post_frames   = int(np.ceil(post_frames/fRate))
   analysisWindowDur = 750 # in ms
   analysisWindowDur = int(np.ceil(analysisWindowDur/fRate))
   duration ='5'

   #Create a huge dictionary with all cells and parameters for each cell
   pathname = info.recordingList.analysispathname[ind]

   ########## Organise stimuli times 
   filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
   filenameCSV = [f for f in glob.glob(filenameCSV)]    
   behData     = pd.read_csv(filenameCSV[0], header=0)
   visTimes    = behData['stimulusOnsetTime'] + behData['trialOffsets']
   rewardTimes = behData['rewardTime'] + behData['trialOffsets']
   choiceTimes = behData['choiceStartTime'] + behData['trialOffsets']

   # Make a variable for rewarded trials
   rewarded    =  behData['rewardTime'].notna() # True for rewarded which also means correct response
   # Calculate reward Time for non-rewarded Trials
   choiceCompleteTimes = behData['choiceCompleteTime'] + behData['trialOffsets']
   diff_time = np.nanmean(rewardTimes -choiceTimes)
   nan_indices = np.isnan(rewardTimes) # unrewarded trials
   rewardTimes[nan_indices] = choiceCompleteTimes[nan_indices] + diff_time

   # Get the stim start times 
   filenameTXT = os.path.join(info.recordingList.path[ind],'twoP') +'\*_imaging_frames.txt'
   filenameTXT= [f for f in glob.glob(filenameTXT)]    
   frame_clock = pd.read_csv(filenameTXT[0],  header= None)
      
   stimFrameTimes    = utils.stim_start_frame_Dual2Psetup (frame_clock, visTimes)
   rewardFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, rewardTimes)

   ########## Organise calcium imaging traces 
   imData = pd.read_pickle (pathname +'imaging-data.pkl')
   fluR      = imData['flu']
   stat      = imData['stat']
   
   flu = utils_laklab.preprocess_flu(fluR, smooth_method='savgol', do_zscore=False, smooth_first=True)         
   flu_zscored = zscore(flu, axis=1) # Z-score for each neuron (across time)
   flu_normalised = zscore (fluR)
   #flu_normalised = mfun.norm_to_zero_one (flu)# Explore dynamics in each sessions

   dffTrace_reward ={} 
   dffTrace_mean_reward ={}
   dffTrace_stimuli ={} 
   dffTrace_mean_stimuli ={}

   ### Get dff values for 2 time windows
   tTypesName = ['Rewarded','Unrewarded']
   tTypes = [(rewarded==True),(rewarded!=True) ]

   for indx, t in enumerate(tTypesName):
      # For reward aligned
      selected_indices = [rewardFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(rewardFrameTimes)))[0]]  
      selected_indices = [value for value in selected_indices if value == value]
      dffTrace_reward[t] = utils.flu_splitter(flu,selected_indices, pre_frames, post_frames)  # Cell x time x trial
      dffTrace_mean_reward[t] = np.mean(dffTrace_reward[t],2) if len(selected_indices)>2 else None # Cell x time
            
      # For stimuli aligned
      selected_indices = [stimFrameTimes[i] for i in np.where(tTypes[indx]==True)[0]]
      selected_indices = [value for value in selected_indices if value == value]
      dffTrace_stimuli[t] = utils.flu_splitter(flu_zscored, selected_indices, pre_frames, post_frames) # Cell x time x trial 
      dffTrace_mean_stimuli[t] = np.mean(dffTrace_stimuli[t],2) if len(selected_indices)>2 else None # Cell x time
               # Heat Plots
   colormap = 'viridis'
   selectedSession = 'WithinSession'
   save_path = info.recordingList.analysispathname[ind]
   analysis_params = ['Rewarded', 'Unrewarded']

   savefigname = 'FirstCheck-heatmap-rewardAligned_flu_' + str(duration[0]) + 'sec'
   pfun.heatmap_sessions(dffTrace_mean_reward, analysis_params, colormap,
                        selectedSession, duration, savefigname, save_path) 
   plt.close()

   savefigname = 'FirstCheck-heatmap-stimuliAligned_fluZscored_' + str(duration[0]) + 'sec'
   pfun.heatmap_sessions(dffTrace_mean_stimuli, analysis_params, colormap,
                        selectedSession, duration, savefigname, save_path) 
   plt.close()
         
   # Mean Plots 
   zscoreRun = False 
   baseline_subtract = [-1.0, 0.0]  # Baseline window: 1 second before stimulus onset

   # Get animal number and session info
   session_name = info.recordingList.sessionName[ind]
   title_prefix = f'{session_name}'

   colormap = ['red', 'black']
   savefigname = 'FirstCheck-mean-rewardAligned_' + str(duration[0]) + 'sec'
   pfun.lineplot_sessions(dffTrace_mean_reward, analysis_params, colormap,
                        duration, zscoreRun, savefigname, save_path, baseline_subtract, 
                        title=f'{title_prefix} - Rewarded vs Unrewarded') 
   plt.close()


CHANGE the animal ID  & selection criteria. Performance works or 2AFC contrast taks.

In [43]:
# Get the list to extract the files for further analysis
animalList= ['MBL014']
info = mfun.analysis(animalList=animalList)
info.recordingList = info.recordingList[
    (info.recordingList['performance'] > 60) &
    (info.recordingList['twoP']) &
    (info.recordingList['duration'] > 20)
    ].reset_index(drop=True)
print('Total Session fits the selection: ' +  str(info.recordingList.shape[0]))

Env: decMaking310
Computer: Huriye Windows
Most likely expRef does not match Z:\MBL014\2025-06-27\1\2025-06-27_1_MBL014_Block.mat: too many indices for array: array is 0-dimensional, but 1 were indexed
Total Session fits the selection: 11


In [ ]:
# Extract information from raw data files - Creates frames.txt, CSV, suite2p & imaging-data.pkl 
csv_reCalculate = True # To calculate the missing CSV
suite2p_reCalculate = True # To calculate the missing suite2p folder
dff_reCalculate = True

logfile = os.path.join(info.rootPath, 'Preprocessing_log_' +  animalList[0] + '.txt')
tee = mfun.Tee(logfile)
try:
    for ind, recordingDate in enumerate(info.recordingList.recordingDate):
        if ind >=0:
            start_time = time.time()
            print(f"\n#### {ind} #### Preprocessing started for : {info.recordingList.sessionName[ind]} ",  end="\n\n")

            ###### STEP 1: PAQ ///// Extract time events from PAQ file   Z:\MBL014\2025-06-13\TwoP\2025-06-13_MBL014_3_paq_imaging_frames.txt
            print(f"Step 1: Processing paq and creating imaging_frames.txt")
            filenamePAQextracted = glob.glob(os.path.join(info.recordingList.path[ind], "**", "*paq_imaging_frames.txt"))
            if len(filenamePAQextracted)>0:
                print('    .txt file has found. Skipping...')
                info.recordingList.loc[ind,'PAQextracted']=1
            else:
                paq_path  = os.path.join(info.recordingList.path[ind], "**", "*.paq")
                paq_files = glob.glob(paq_path, recursive=True)
                if len(paq_files) == 1:
                    try:
                        extract_paq_data_frame(paq_files[0])
                        info.recordingList.loc[ind,'PAQextracted']=1
                    except KeyError as e:
                        info.recordingList.loc[ind,'PAQextracted']=0
                        print(f"Error processing {paq_files[0]}: {e}")
                elif len(paq_files) > 1:
                    print(f"    Multiple PAQ files found for : {info.recordingList.sessionName[ind]}")
                    print(filenamePAQextracted)
                    info.recordingList.loc[ind,'PAQextracted']=0
                    continue 
                else:
                    print(f"    No PAQ files found for : {info.recordingList.sessionName[ind]}")
                    info.recordingList.loc[ind,'PAQextracted']=0
                    continue 

            ##### check in analysis folder for paq-data.pkl
            filenamePAQdata = glob.glob(os.path.join(info.recordingList.analysispathname[ind], "paq-data.pkl"))
            if len(filenamePAQdata) > 0:
                print('    paq-data.pkl file has found. Skipping...')
                info.recordingList.loc[ind,'PAQdataFound']=1
            else:
                try: 
                    if not os.path.exists(info.recordingList.analysispathname.iloc[ind]) :
                        os.makedirs(info.recordingList.analysispathname.iloc[ind])

                    print('    calculating paq-data.pkl file ...')
                    paq_path  = os.path.join(info.recordingList.path[ind], "**", "*.paq")
                    paq_files = glob.glob(paq_path, recursive=True)
                    paq_data = utils.paq_read( file_path=paq_files[0], plot=True, save_path=info.recordingList.analysispathname[ind])
                    filenamePAQ_analysis = os.path.join(info.recordingList.analysispathname[ind], 'paq-data.pkl')
                    with open(filenamePAQ_analysis, 'wb') as f:
                        pickle.dump(paq_data, f)
                    del paq_data
                    info.recordingList.loc[ind,'PAQextracted']=1
                except Exception as e:
                    print(f"Error processing {paq_files[0]}: {e}")
                    info.recordingList.loc[ind,'PAQextracted']=0

            ### >>> We have PAQ file properly & now can do analysis on this recording - Lets create a folder for analysis preprocessing files
            if info.recordingList.loc[ind,'PAQextracted'] !=1:
                info.recordingList.loc[ind,'CSVcreated']=0
                continue 
        
            ###### STEP 2: CSV ///// create CSV for behaviour outcome
            print('Step 2: Creating CSV file')
            filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
            e_filenameCSV = [f for f in glob.glob(filenameCSV)]
            if len(e_filenameCSV)==1:
                print('    CorrectedeventTimes.csv file has found. Skipping...')
                info.recordingList.loc[ind,'CSVcreated']=1
                info.recordingList.loc[ind,'CSVpath'] = filenameCSV
            else:
                if  (not csv_reCalculate) & (info.recordingList.loc[ind,'PAQextracted'] == 0):
                    info.recordingList.loc[ind,'CSVcreated']=0
                    info.recordingList.loc[ind,'CSVpath'] = filenameCSV
                else:
                    try: 
                        info.recordingList.loc[ind,'CSVpath'] = filenameCSV
                        filenameTimeline = [f for f in glob.glob(info.recordingList.filepathname[ind]+ '\\' + info.recordingList.sessionName[ind] + '_Timeline.mat')]
                        if  (len(filenameTimeline)>0):
                            sessionProfile ='Grating2AFC'
                        else:
                            sessionProfile ='Grating2AFC_noTimeline'
                            
                        # Get behaviour trial data from Block.mat file
                        data = eng.getBehavData(info.recordingList.sessionName[ind],sessionProfile)

                        # Apply correction based on the weights to match the eventTimes
                        if (len(filenameTimeline)>0) :
                            print('    Aligning time events:', end="", flush=True)
                            # Get weights to convert from probe to behavioural timebase
                            twoPpath = info.recordingList.path[ind] + '\\TwoP'
                            sessionName = info.recordingList.sessionName[ind]
                            figsavepath = info.recordingList.analysispathname[ind]
                            dataCorrected, variance = eng.applySubtractionCorrection (data, twoPpath ,sessionName, True, figsavepath, nargout=2)
                            eng.close('all', nargout=0) 
                            info.recordingList.loc[ind,'variance'] = variance

                            # Apply correction to the signal
                            data = dataCorrected
                                
                        # Save the file
                        eng.writetable(data, filenameCSV, nargout=0)
                        print("Completed.")
                        info.recordingList.loc[ind,'CSVcreated']=1
                    except Exception as e:
                        print(str(ind) + ' !!!!! FAILED: Creating CSV: ' + info.recordingList.sessionName[ind])
                        info.recordingList.loc[ind,'CSVcreated']=0

            ### >>> Only if we have CSV file properly - then we are ready for imaging data analysis
            if info.recordingList.loc[ind,'CSVcreated'] !=1:
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                info.recordingList.loc[ind, 'dffcreated'] = 0
                continue

            ###### STEP 3: check recording quality - extract cells ROIS with suite2p 
            print('Step 3: check suite2p')
            twoP_dir = os.path.dirname(info.recordingList.imagingTiffFileNames[ind])
            tiff_path    = info.recordingList.imagingTiffFileNames[ind]
            filenameTiff = glob.glob(os.path.join(tiff_path, "*Ch2.tif"))
            if len(filenameTiff) == 0:
                print(f"TIFF missing/invalid for index {ind}: {tiff_path}")
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                continue

            ref_dir = os.path.join(tiff_path, "References")
            ref_matches = glob.glob(os.path.join(ref_dir, "*-Ch2-16bit-Reference.tif"))
            
            #---------   Average FOV   ----------
            if not ref_matches:
                print(f"    No reference average found in {ref_dir}")
            else:
                ref_file = ref_matches[0]  # take the first match
                # destination in analysis folder
                analysis_dir = info.recordingList.analysispathname.iloc[ind]
                avg_path = os.path.join(
                    analysis_dir,
                    os.path.basename(ref_file))
                avg_path = avg_path.replace(".tif", ".png") 

                if not os.path.exists(avg_path):
                    mfun.save_png_with_contrast(ref_file, avg_path, also_save_16bit=False)
                    #shutil.copy2(ref_file, avg_path)  # copy with metadata
                    print(f"    Copied reference avg to {avg_path}")
                else:
                    print(f"    Reference avg already exists. Skipping...")

            # --------- SUITE2P CHECK / RUN ----------
            suite2p_dir = os.path.join(tiff_path, "suite2p")
            if  (not suite2p_reCalculate) and os.path.isdir(suite2p_dir):
                info.recordingList.loc[ind, 'suite2Pcreated'] = 1
                print(f"    suite2p found. Skipping..")
            elif suite2p_reCalculate:
                print(f"    suite2p not found. Running extraction for: {tiff_path}")
                try:
                    filenameENV = glob.glob(os.path.join(tiff_path, "*.env"))
                    imagingDetails = mfun.parse_pv_env(filenameENV[0])
                    mfun.suite2p_extraction(tiff_path,
                                            ops_yaml_path=info.ops_yaml_path,
                                            imagingDetails=imagingDetails)
                # mfun.suite2p_extraction(tiff_path)
                    info.recordingList.loc[ind, 'suite2Pcreated'] = 1
                except Exception as e:
                    print(f"    suite2p_extraction failed for {tiff_path}: {e}")
                # set flag based on result on disk
                info.recordingList.loc[ind, 'suite2Pcreated'] = 1
            else:
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                print(f"    suite2p not found and not calculated.")

            # ###### STEP 4: create imaging files
            print('Step 4: calculate & create imaging.pkl')

            # Read suite2p
            s2p_path = os.path.join(info.recordingList.path[ind], 'TwoP', f"{info.recordingList.recordingDate[ind]}_t-001", 'suite2p', 'plane0')
            filenameDFF = os.path.join(info.recordingList.analysispathname[ind], 'imaging-data.pkl')
            if (not dff_reCalculate) and os.path.exists(filenameDFF):
                print(f"    imaging-data.pkl already exists. Skipping...")
                info.recordingList.loc[ind, 'dffcreated'] = 1

            elif dff_reCalculate & (os.path.exists(s2p_path)) & (info.recordingList.loc[ind, 'suite2Pcreated'] == 1):
                print('    Loading suite2p data...')
                ops = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True)
                ops = ops.item()
                FrameNums = ops['frames_per_file']
                filelist = ops['filelist']
                isCell = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)

                # Load the suite2p
                flu_raw_subtracted, spks, stat = utils.s2p_loader(s2p_path)
                flu = utils.dfof2(flu_raw_subtracted)

                # Cut each session & save it in the analysis-session folder
                imaging_data = {
                    "n_frames": FrameNums,
                    "flu": flu,
                    "spks": spks,
                    "stat": stat,
                }
                
                print(f"{flu.shape[0]} cells; {flu.shape[1]} frames")
                with open(filenameDFF, 'wb') as f:
                    pickle.dump(imaging_data, f)
                print(f"    Successfully saved imaging data.")
                create_FOV_withSelectedCells(isCell, ops, s2p_path)
                print(f"    Successfully saved FOV with selected cells.")
                calculateHeatmapsForRewardedStimulus()
                print(f"    Successfully saved heatmaps.")
                info.recordingList.loc[ind, 'dffcreated'] = 1
            else:
                print(f"Suite2p path not found: {s2p_path}")
                info.recordingList.loc[ind, 'dffcreated'] = 0   
            end_time = time.time()  
            elapsed = time.time() - start_time
            print(f"{ind} Successfully completed in {elapsed:.1f} seconds\n")   

finally:
    tee.close()   # important to restore stdout/stderr and close file

# display the output
print( "Behaviour trial data extraction completed: " + 
      str(info.recordingList['dffcreated'].sum()) +"/" + str(info.recordingList.shape[0]))


#### 0 #### Preprocessing started for : 2025-06-12_1_MBL014 

Step 1: Processing paq and creating imaging_frames.txt
    .txt file has found. Skipping...
    paq-data.pkl file has found. Skipping...
Step 2: Creating CSV file
    CorrectedeventTimes.csv file has found. Skipping...
Step 3: check suite2p
    Reference avg already exists. Skipping...
    suite2p not found. Running extraction for: Z:\MBL014\2025-06-12\TwoP\2025-06-12_t-001
29.873733883942098
Z:\MBL014\2025-06-12\TwoP\2025-06-12_t-001
CuPy version: 13.6.0
CUDA device: b'NVIDIA GeForce RTX 3080'
Ops useGPU: True
{'data_path': 'Z:\\MBL014\\2025-06-12\\TwoP\\2025-06-12_t-001', 'tiff_list': ['Z:\\MBL014\\2025-06-12\\TwoP\\2025-06-12_t-001\\2025-06-12_t-001_Cycle00001_Ch2.tif'], 'save_path': 'Z:\\MBL014\\2025-06-12\\TwoP\\2025-06-12_t-001'}
FOUND BINARIES AND OPS IN ['Z:\\MBL014\\2025-06-12\\TwoP\\2025-06-12_t-001\\suite2p\\plane0\\ops.npy']
removing previous detection and extraction files, if present
>>>>>>>>>>>>>>>>>>>>> PLANE

In [ ]:
#10> Save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

# Save table as CSV
recordingList = info.recordingList
recordingList.to_csv( info.analysisPath +'\\recordingList.csv', index=False)